In [1]:
System.out.println("Hello World!")

Hello World!


In [3]:
%%jars
samples/app/target/lib

Found 164 jar files.
Add /home/dsyer/dev/scratch/grpc-reflect/samples/app/target/lib/netty-resolver-4.1.124.Final.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/samples/app/target/lib/spring-boot-test-autoconfigure-3.5.4.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/samples/app/target/lib/jakarta.activation-api-2.1.3.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/samples/app/target/lib/micrometer-core-1.15.3.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/samples/app/target/lib/spring-test-6.2.10.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/samples/app/target/lib/assertj-core-3.27.3.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/samples/app/target/lib/accessors-smart-2.5.2.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/samples/app/target/lib/junit-4.13.2.jar to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/samples/app/target/lib/netty-codec-http-4.1.124.Final.jar to classpath
Add /home/dsyer/dev/scratch

In [1]:
%dependency /add com.fasterxml.jackson.dataformat:jackson-dataformat-protobuf:2.20.0

Adding dependency com.fasterxml.jackson.dataformat:jackson-dataformat-protobuf:2.20.0


In [ ]:
%classpath samples/app/target/test-classes/
%classpath samples/app/target/classes/
%classpath samples/webflux/target/classes/
%classpath samples/webflux/target/test-classes/

Add /home/dsyer/dev/scratch/grpc-reflect/app/target/test-classes to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/app/target/classes to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/webflux/target/classes to classpath
Add /home/dsyer/dev/scratch/grpc-reflect/webflux/target/test-classes to classpath


In [4]:
import org.springframework.grpc.sample.proto.*;

In [5]:
var bindable = new SimpleGrpc.SimpleImplBase() {};

In [83]:
import io.grpc.protobuf.ProtoServiceDescriptorSupplier;
import io.grpc.protobuf.ProtoFileDescriptorSupplier;
import io.grpc.*;

In [9]:
bindable.bindService().getServiceDescriptor().getMethods().forEach(method -> {
	System.out.println("Method: " + method);
	System.out.println("Method: " + method.getFullMethodName());
	System.out.println("Input: " + ((ProtoServiceDescriptorSupplier)method.getSchemaDescriptor()).getServiceDescriptor().findMethodByName(method.getBareMethodName()).getRequestMarshaller().getMessageClass());
	System.out.println("Input: " + ((ProtoServiceDescriptorSupplier)method.getSchemaDescriptor()).getServiceDescriptor().findMethodByName(method.getBareMethodName()).getInputType().getFullName());
	System.out.println("File: " + ((ProtoServiceDescriptorSupplier)method.getSchemaDescriptor()).getFileDescriptor());
	System.out.println("Schema: " + ((ProtoServiceDescriptorSupplier)method.getSchemaDescriptor()).getServiceDescriptor().findMethodByName(method.getBareMethodName()).toProto());
});

CompilerException: 

In [39]:
import io.grpc.protobuf.services.ProtoReflectionServiceV1;
import io.grpc.reflection.v1.ServerReflectionRequest;
import io.grpc.reflection.v1.ServerReflectionResponse;
import io.grpc.reflection.v1.ServiceResponse;
import io.grpc.reflection.v1.ListServiceResponse;
import io.grpc.reflection.v1.FileDescriptorResponse;

In [10]:
var request = ServerReflectionRequest.newBuilder()
	.setListServices("")
	.build();


In [38]:
var response = ServerReflectionResponse.newBuilder()
	.setValidHost(request.getHost())
	.setListServicesResponse(ListServiceResponse.newBuilder().addService(ServiceResponse.newBuilder().setName("Foo").build()).build())
	.setOriginalRequest(request).build();
System.out.println("Response: " + response);

Response: original_request {
  list_services: ""
}
list_services_response {
  service {
    name: "Foo"
  }
}



In [47]:
var method = bindable.bindService().getServiceDescriptor().getMethods().stream()
	.filter(m -> m.getBareMethodName().equals("SayHello"))
	.findFirst()
	.orElseThrow(() -> new RuntimeException("Method not found"));
var fileDescriptor = ((ProtoServiceDescriptorSupplier)method.getSchemaDescriptor()).getFileDescriptor();

In [50]:
var response = ServerReflectionResponse.newBuilder()
	.setValidHost(request.getHost())
	.setOriginalRequest(request)
	.setFileDescriptorResponse(FileDescriptorResponse.newBuilder()
		.addFileDescriptorProto(fileDescriptor.toProto().toByteString())
		.build())
	.build();
System.out.println("Response: " + response);

Response: original_request {
  list_services: ""
}
file_descriptor_response {
  file_descriptor_proto: "\n\vhello.proto\"\034\n\fHelloRequest\022\f\n\004name\030\001 \001(\t\"\035\n\nHelloReply\022\017\n\amessage\030\001 \001(\t2a\n\006Simple\022(\n\bSayHello\022\r.HelloRequest\032\v.HelloReply\"\000\022-\n\vStreamHello\022\r.HelloRequest\032\v.HelloReply\"\0000\001B:\n%org.springframework.grpc.sample.protoB\017HelloWorldProtoP\001b\006proto3"
}



In [63]:
import com.google.protobuf.Message;

In [ ]:
var call = new ServerCall() {
	@Override
	public void request(int numMessages) {
		System.out.println("Requesting " + numMessages + " messages");
	}

	@Override
	public void sendMessage(Object message) {
		System.out.println("Sending message: " + message);
	}

	@Override
	public void close(Status status, Metadata trailers) {
		System.out.println("Call closed with status: " + status);
	}

	@Override
	public boolean isCancelled() {
		return false;
	}

	@Override
	public void sendHeaders(Metadata headers) {
		System.out.println("Sending headers: " + headers);
	}

	@Override
	public MethodDescriptor getMethodDescriptor() {
		return method;
	}
};

In [93]:
bindable.bindService().getMethod("Simple/SayHello").getServerCallHandler()

io.grpc.stub.ServerCalls$UnaryServerCallHandler@3a85ab0d

In [88]:
var listener = bindable.bindService().getMethod("Simple/SayHello").getServerCallHandler().startCall(call, null);

Requesting 2 messages


In [94]:
listener

io.grpc.stub.ServerCalls$UnaryServerCallHandler$UnaryServerCallListener@16b5a0b4

In [95]:
listener.onMessage(HelloRequest.newBuilder().setName("World").build());

In [96]:
listener.onHalfClose();

Call closed with status: Status{code=UNIMPLEMENTED, description=Method Simple/SayHello is unimplemented, cause=null}


In [92]:
listener.onComplete();

In [99]:
var listener = ProtoReflectionServiceV1.newInstance().bindService().getMethod("grpc.reflection.v1.ServerReflection/ServerReflectionInfo").getServerCallHandler().startCall(call, null);

EvalException: Cannot invoke "io.grpc.Server.getImmutableServices()" because "server" is null

In [122]:
var a = 0.02;
var X = 1332.72;
var n = 24;
var y = 69.07;
a - ((a*X - y)*Math.pow(1 + a, n) + y)/(X*Math.pow(1+a, n) + n*(a*X -y)*Math.pow(1+a, n-1));

0.01842637013657085

In [128]:
a = a - ((a*X - y)*Math.pow(1 + a, n) + y)/(X*Math.pow(1+a, n) + n*(a*X -y)*Math.pow(1+a, n-1));
a

0.018245871996787587

In [129]:
System.out.println(String.format("a = %f, 12a = %f, (1+a)^12 - 1 = %f", a, 12*a, Math.pow(1 + a, 12) - 1));
System.out.println(String.format("(ny/X - 1)/n*12 = %f", (n*y/X - 1)/n*12));

a = 0.018246, 12a = 0.218950, (1+a)^12 - 1 = 0.242315
(ny/X - 1)/n*12 = 0.121916


In [130]:
a*X/(1 - Math.pow(1 + a, -n))

69.07000000000023

In [138]:
var a = .239/12;
var X = 3066.72;
var n = 24;
a*X/(1 - Math.pow(1 + a, -(n+4)))

143.94863584148885